In [20]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import fiftyone as fo
import fiftyone.zoo as foz
import os

In [26]:
os.environ["FIFTYONE_DEFAULT_DATASET_DIR"] = "Z:/open_images_v7"
fo.config.dataset_zoo_dir = "Z:/open_images_v7"

In [27]:
os.makedirs(custom_download_dir, exist_ok=True)

### Load Model

**Load Food101**

In [ ]:
food101_dataset = tfds.load('food101', split=['train', 'validation'], as_supervised=True)

# Print a sample from Food101 to confirm it's loaded correctly
for image, label in food101_dataset['train'].take(1):
    print("Image shape:", image.shape)
    print("Label:", label.numpy().decode("utf-8"))  # Assuming labels are strings


**Load Open Images V7**

In [31]:
food_classes = ["Egg (Food)", "Fast food", "Food", "Seafood",]

In [ ]:
open_v7_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    splits=["train", "validation"],
    classes=food_classes,
    max_samples=10000  # Adjust this number as needed
)

**Check Classes in the Dataset**

In [12]:
def load_and_preprocess_data(dataset):
    images = []
    labels = []
    for image, label in dataset:
        # Resize image to 224x224
        image = tf.image.resize(image, (224, 224))
        # Normalize pixel values
        image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

### Combine Datasets

In [ ]:
x_food101, y_food101 = load_and_preprocess_data(food101_dataset['train'])
x_open_v7, y_open_v7 = load_and_preprocess_data(open_v7_dataset['train'])

x_train = np.concatenate([x_food101, x_open_v7])
y_train = np.concatenate([y_food101, y_open_v7])

### Model architecture

In [ ]:
num_classes = len(np.unique(y_train))

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

**Training**

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

**Inference**

In [ ]:
def predict_food(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    
    return predicted_class

### Nutrition data integration

In [ ]:
nutrition_data = pd.read_csv("Nutrition-Data/nutrients_csvfile.csv")

def get_nutrition_info(food_item):
    try:
        nutrition = nutrition_data[nutrition_data['food_item'] == food_item].iloc[0]
        return {
            'calories': nutrition['calories'],
            'protein': nutrition['protein'],
            'carbs': nutrition['carbs'],
            'fat': nutrition['fat']
        }
    except IndexError:
        return {
            'calories': 0,
            'protein': 0,
            'carbs': 0,
            'fat': 0
        }

**API preparation**

In [ ]:
def food_recognition_and_nutrition(image_path):
    predicted_class = predict_food(image_path)
    food_item = class_labels[predicted_class]  # You'll need to create a mapping of class indices to food names
    nutrition_info = get_nutrition_info(food_item)
    
    return {
        'food_item': food_item,
        'nutrition_info': nutrition_info
    }

**Function for future API integration**

In [ ]:
def get_nutrition_info_from_api(food_item):
    api_url = "https://api.example.com/nutrition"
    params = {"food": food_item}
    
    try:
        response = requests.get(api_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return {
                'calories': data['calories'],
                'protein': data['protein'],
                'carbs': data['carbs'],
                'fat': data['fat']
            }
        else:
            return {
                'calories': 0,
                'protein': 0,
                'carbs': 0,
                'fat': 0
            }
    except requests.RequestException:
        return {
            'calories': 0,
            'protein': 0,
            'carbs': 0,
            'fat': 0
        }

### TFLITE Model

In [ ]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('food_recognition_model_v2.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved successfully.")